<a href="https://colab.research.google.com/github/cksdlakstp12/Fire_Safety_AI_Competition/blob/main/FireAIModelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import pickle

import time
from datetime import datetime, timedelta
from calendar import monthrange

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


출동 빈도가 1 이상인 행만 추출해 새로운 데이터 셋 제작

In [ ]:
GRID_DATASET_DIR = "/content/drive/MyDrive/datasets/fire_fighting_safe"

In [ ]:
df = pd.read_csv(f"{GRID_DATASET_DIR}/grid_dataset.csv")

all_YMD_list = sorted(map(str, list(set(df["YMD"][:])))) # 데이터에 있는 모든 날짜를 담은 리스트
print(all_YMD_list)

df = df[df.apply(lambda x: x["DSP_CNT_IDEX"] > 0, axis=1)] # 출동 빈도가 1 이상인 데이터만 추출
df.to_csv(f"{GRID_DATASET_DIR}/grid_dataset_filtered_by_DSP_CNT.csv") # 추출된 데이터 저장

['20200101', '20200102', '20200103', '20200104', '20200105', '20200106', '20200107', '20200108', '20200109', '20200110', '20200111', '20200112', '20200113', '20200114', '20200115', '20200116', '20200117', '20200118', '20200119', '20200120', '20200121', '20200122', '20200123', '20200124', '20200125', '20200126', '20200127', '20200128', '20200129', '20200130', '20200201', '20200202', '20200203', '20200204', '20200205', '20200206', '20200207', '20200208', '20200209', '20200210', '20200211', '20200212', '20200213', '20200214', '20200215', '20200216', '20200217', '20200218', '20200219', '20200220', '20200221', '20200222', '20200223', '20200224', '20200225', '20200226', '20200227', '20200229', '20200301', '20200302', '20200303', '20200304', '20200305', '20200306', '20200307', '20200308', '20200309', '20200310', '20200311', '20200312', '20200313', '20200314', '20200315', '20200316', '20200317', '20200318', '20200319', '20200320', '20200321', '20200322', '20200323', '20200324', '20200325', '20

In [ ]:
df

,GRID_ID,GRID_X_AXIS,GRID_Y_AXIS,TIME,YMD,STDG_CD,FP_IDEX,OLD_FP_IDEX,RELIFPLC_DWL_IDEX,RELIFPLC_ROAD_IDEX,RELIFPLC_INDUST_IDEX,RELIFPLC_NTR_IDEX,RELIFPLC_ETC_IDEX,DSP_CNT_IDEX,DISEASE_DSP_IDEX,DISEASE_EXCL_DSP_IDEX
28,38545254,385475,525475,0,20200101,4213000000,7.57,0.49,1,0,0,0,0,1,1,0
73,39345264,393475,526475,0,20200101,4213000000,1991.34,79.17,1,0,0,0,0,2,1,1
84,39445284,394475,528475,0,20200101,4213000000,1515.30,80.44,1,0,0,0,0,1,1,0
106,39645274,396475,527475,0,20200101,4213000000,389.21,63.22,0,0,1,0,0,1,0,1
216,39445284,394475,528475,1,20200101,4213000000,1539.88,79.37,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768431,39945214,399475,521475,20,20201230,4213000000,11.99,1.68,0,0,1,0,0,1,1,0
1768574,39445264,394475,526475,21,20201230,4213000000,4563.22,379.53,0,1,0,0,0,1,0,1
1768632,39845244,398475,524475,21,20201230,4213000000,1245.73,56.89,1,0,0,0,0,1,0,1
1768881,39445374,394475,537475,22,20201230,4213000000,0.00,0.00,0,0,0,0,0,2,0,2


입력받은 날로부터 설정된 간격까지의 날을 추출해 리스트에 저장

In [ ]:
stride = 7

mindate = 20200101

input_YMD_list = all_YMD_list

In [ ]:
def calc_stride_range_YMD_list(stride, mindate, input_YMD_list):

  stride_range_YMD_list = list()

  for input_YMD in input_YMD_list:

    Y = int(input_YMD[:4])
    M = int(input_YMD[4:6])
    D = int(input_YMD[6:])

    YMD = datetime(Y, M, D)

    stride_YMD_list = list()
    for n in range(1, stride + 1):
      calc_YMD = YMD + timedelta(days=n*-1)

      zero_padding_MD = [calc_YMD.month, calc_YMD.day]

      for i, MD in enumerate(zero_padding_MD):
        if MD // 10 == 0:
          zero_padding_MD[i] = f"0{MD}"

      calc_YMD = f"{calc_YMD.year}{zero_padding_MD[0]}{zero_padding_MD[1]}"

      if int(calc_YMD) >= mindate:
        stride_YMD_list.append(int(calc_YMD))
  
    stride_range_YMD_list.append(stride_YMD_list)
    
  stride_range_YMD_list = [i for i in stride_range_YMD_list if i]
  return stride_range_YMD_list

In [ ]:
print(calc_stride_range_YMD_list(stride, mindate, input_YMD_list))

[[20200101], [20200102, 20200101], [20200103, 20200102, 20200101], [20200104, 20200103, 20200102, 20200101], [20200105, 20200104, 20200103, 20200102, 20200101], [20200106, 20200105, 20200104, 20200103, 20200102, 20200101], [20200107, 20200106, 20200105, 20200104, 20200103, 20200102, 20200101], [20200108, 20200107, 20200106, 20200105, 20200104, 20200103, 20200102], [20200109, 20200108, 20200107, 20200106, 20200105, 20200104, 20200103], [20200110, 20200109, 20200108, 20200107, 20200106, 20200105, 20200104], [20200111, 20200110, 20200109, 20200108, 20200107, 20200106, 20200105], [20200112, 20200111, 20200110, 20200109, 20200108, 20200107, 20200106], [20200113, 20200112, 20200111, 20200110, 20200109, 20200108, 20200107], [20200114, 20200113, 20200112, 20200111, 20200110, 20200109, 20200108], [20200115, 20200114, 20200113, 20200112, 20200111, 20200110, 20200109], [20200116, 20200115, 20200114, 20200113, 20200112, 20200111, 20200110], [20200117, 20200116, 20200115, 20200114, 20200113, 202001

위에서 얻은 YDM 리스트를 이용해서 시간대별 가장 출동 빈도가 큰 GRID_ID를 추출함

In [ ]:
# 리스트를 len(arr) // n 만큼 분할해줌
def list_chuck(arr, n):
  return [arr[i:i+n] for i in range(0, len(arr), n)]

In [ ]:
def get_filtered_grid_ids(division_cnt, YMD_list):
  time_list = list(map(int, range(24)))
  time_list = list_chuck(time_list, division_cnt)

  # DataFrame에서 원하는 날짜와 시간에 해당되는 행만 추출함
  filtered_df_by_TIME_and_YMD = list()
  for YMD in YMD_list:
    result_df_list = list()

    filtered_df_by_YMD = df[df.apply(lambda x: x["YMD"] in YMD, axis=1)]

    for times in time_list:
      try:
        filtered_df_by_time = filtered_df_by_YMD[filtered_df_by_YMD.apply(lambda x: x["TIME"] in times, axis=1)]
      except KeyError:
        continue
      result_df_list.append(filtered_df_by_time)
    filtered_df_by_TIME_and_YMD.append(result_df_list)

  # 추출된 DataFrame에서 DSP_CNT_IDEX의 합이 가장 큰 GRID_ID만 추출함
  filtered_grid_ids = list()
  for filtered_datas in filtered_df_by_TIME_and_YMD:
    result_grid_ids = list()

    for filtered_data in filtered_datas:
      result_grid_id = filtered_data.groupby("GRID_ID")["DSP_CNT_IDEX"].sum().reset_index().max()["GRID_ID"]

      for _ in range(division_cnt):
        result_grid_ids.append(result_grid_id)
    filtered_grid_ids.append(result_grid_ids)
  return filtered_grid_ids

예측 정확도 측정

In [ ]:
YMD_list = calc_stride_range_YMD_list(stride, mindate, input_YMD_list)

for division_cnt in [1,2,3,4,6,8,12,24]:
  filtered_grid_ids = get_filtered_grid_ids(division_cnt, YMD_list)
  filtered_grid_ids.insert(0, [39545254]*24)

  pred_result_list = list()
  for input_YMD, filtered_grid_id in zip(input_YMD_list, filtered_grid_ids):
    for time, grid_id in zip(range(24), filtered_grid_id):
      pred_result_list.append(tuple([int(input_YMD), time, grid_id]))

  res = [] # 결과 리스트
  for (ymd, t, grid_id) in pred_result_list: # YMD, TIME 상에 해당 GRID_ID의 DSP_CNT_IDEX가 1 이상인지를 확인하는 로직
    v = (df.loc[(df["YMD"] == ymd) & (df["TIME"] == t) & (df["GRID_ID"] == grid_id)]["DSP_CNT_IDEX"] >= 1).any()# 만약 특정 YMD, TIME, GRID_ID의 DSP_CNT_IDEX가 1 이상이면 1, 또는 0
    res.append(v) # 결과에 추가
  print(f"예측 정확도 (at division_cnt : {division_cnt}):", np.mean(np.array(res))) # 결과의 평균 정확도를 출력

예측 정확도 (at division_cnt : 1): 0.011652542372881356
예측 정확도 (at division_cnt : 2): 0.007415254237288136
예측 정확도 (at division_cnt : 3): 0.004119585687382297
예측 정확도 (at division_cnt : 4): 0.0030602636534839923
예측 정확도 (at division_cnt : 6): 0.0023540489642184556
예측 정확도 (at division_cnt : 8): 0.00211864406779661
예측 정확도 (at division_cnt : 12): 0.0018832391713747645
예측 정확도 (at division_cnt : 24): 0.0014124293785310734


In [ ]:
max = [0, 0, 0]

for stride in range(1, 31):

  YMD_list = calc_stride_range_YMD_list(stride, mindate, input_YMD_list)

  for division_cnt in [1, 2, 3, 4, 6, 8, 12, 24]:
    filtered_grid_ids = get_filtered_grid_ids(division_cnt, YMD_list)
    filtered_grid_ids.insert(0, [39545254]*24)

    pred_result_list = list()
    for input_YMD, filtered_grid_id in zip(input_YMD_list, filtered_grid_ids):
      for time, grid_id in zip(range(24), filtered_grid_id):
        pred_result_list.append(tuple([int(input_YMD), time, grid_id]))

    res = [] # 결과 리스트
    for (ymd, t, grid_id) in pred_result_list: # YMD, TIME 상에 해당 GRID_ID의 DSP_CNT_IDEX가 1 이상인지를 확인하는 로직
      v = (df.loc[(df["YMD"] == ymd) & (df["TIME"] == t) & (df["GRID_ID"] == grid_id)]["DSP_CNT_IDEX"] >= 1).any()# 만약 특정 YMD, TIME, GRID_ID의 DSP_CNT_IDEX가 1 이상이면 1, 또는 0
      res.append(v) # 결과에 추가
    print(f"예측 정확도 (at stride and division_cnt : {stride} and {division_cnt}):", np.mean(np.array(res))) # 결과의 평균 정확도를 출력

    if max[2] < np.mean(np.array(res)):
      max = [stride, division_cnt, np.mean(np.array(res))]


with open(f"{GRID_DATASET_DIR}/results.pkl", "wb") as fp:
  pickle.dump(max, fp)

예측 정확도 (at stride and division_cnt : 4 and 12): 0.0024717514124293787
예측 정확도 (at stride and division_cnt : 4 and 24): 0.002236346516007533
예측 정확도 (at stride and division_cnt : 5 and 1): 0.0160075329566855
예측 정확도 (at stride and division_cnt : 5 and 2): 0.009651600753295669
예측 정확도 (at stride and division_cnt : 5 and 3): 0.005532015065913371
예측 정확도 (at stride and division_cnt : 5 and 4): 0.0036487758945386063
예측 정확도 (at stride and division_cnt : 5 and 6): 0.002824858757062147
예측 정확도 (at stride and division_cnt : 5 and 8): 0.0024717514124293787
예측 정확도 (at stride and division_cnt : 5 and 12): 0.0023540489642184556
예측 정확도 (at stride and division_cnt : 5 and 24): 0.0018832391713747645
예측 정확도 (at stride and division_cnt : 6 and 1): 0.012947269303201506
예측 정확도 (at stride and division_cnt : 6 and 2): 0.00800376647834275
예측 정확도 (at stride and division_cnt : 6 and 3): 0.004825800376647834
예측 정확도 (at stride and division_cnt : 6 and 4): 0.003413370998116761
예측 정확도 (at stride and division_cnt : 6 and

KeyboardInterrupt: ignored

RESULT

결과를 보았을 때 최대 정확도가 3.1%였다.

반면 전체에서 빈도수가 가장 높은 그리드를 전부 채우니 정확도가 11%가 나왔다.

데이터가 통계 rule based ai를 통해서는 높게 나오지 않는 듯해 보인다.